In [7]:
import numpy as np
import pandas as pd
from scipy import stats
import sys
import argparse
import h5py
from scipy.stats import t as student_t
from statsmodels.stats import multitest as mt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [8]:
vars_df = pd.read_pickle('male_vars_testB.pkl')

with open('varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
with open('varsVARS.txt') as f:
    varsVARS = f.readlines()
    varsVARS = [l.strip('\n\r') for l in varsVARS]
    varsVARS = np.array(varsVARS)
# vars_categories = np.loadtxt("vars_categories.txt", dtype=str, delimiter='\n')
with open('vars_categories.txt') as f:
    vars_categories = f.readlines()
    vars_categories = [l.strip('\n\r') for l in vars_categories]
    vars_categories = np.array(vars_categories)

In [9]:
vars_df.head()

,modality,bonf,fdr,dataframe
0,T1_nonlinear,5.443326,4.755399,idx ...
1,T1_linear,5.443326,NaN,idx ...
2,jacobian,5.443326,NaN,idx ...
3,vbm,5.443326,4.996503,idx ...
4,T2_nonlinear,5.443326,5.612791,idx ...


In [10]:
vars_df.iloc[0].dataframe

,idx,names,Categories,pearson_r,t_test_statistic,p_values_corrected,p_values,abs_pearson_r,log_p_values,log_p_values_corrected
0,0,Ethnic background (0.0),Ethnic Background,0.014256,0.664910,1.000000,0.506179,0.014256,0.295696,-0.000000
1,1,Ethnic background (1.0),Ethnic Background,-0.029316,0.578459,1.000000,0.563289,0.029316,0.249269,-0.000000
2,2,Ethnic background (2.0),Ethnic Background,0.060473,1.530274,0.976350,0.126445,0.060473,0.898099,0.010394
3,3,Genotype measurement batch (0.0),Genetic Markers,-0.026032,1.203263,1.000000,0.229008,0.026032,0.640149,-0.000000
4,4,Heterozygosity (0.0),Genetic Markers,-0.010863,0.501956,1.000000,0.615750,0.010863,0.210595,-0.000000
...,...,...,...,...,...,...,...,...,...,...
13872,17504,Substances taken for depression (0.1),Mental Health,0.098974,1.798598,0.887889,0.073004,0.098974,1.136651,0.051641
13873,17506,Activities undertaken to treat depression (0.1),Mental Health,-0.099825,1.598931,0.949901,0.111079,0.099825,0.954368,0.022322
13874,17507,Manifestations of mania or irritability (0.1),Mental Health,-0.047695,0.791841,1.000000,0.429136,0.047695,0.367405,-0.000000
13875,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.029614,1.205283,1.000000,0.228266,0.029614,0.641558,-0.000000


In [11]:
current_mods = vars_df.modality.values
modality_names = [
    'T1 Nonlinear', 'T1 Linear', 'Jacobian', 'VBM', 'T2 Nonlinear', 'T2 Lessions', 'SWI', 'rsfMRI-0', 'rsfMRI-1',
    'rsfMRI-2', 'rsfMRI-3', 'rsfMRI-4', 'rsfMRI-5', 'rsfMRI-6', 'rsfMRI-7', 'rsfMRI-8', 'rsfMRI-9', 'rsfMRI-10', 
    'rsfMRI-11', 'rsfMRI-12', 'rsfMRI-13', 'rsfMRI-14', 'rsfMRI-15', 'rsfMRI-16', 'rsfMRI-17', 'rsfMRI-18', 'rsfMRI-19',
    'rsfMRI-20', 'rsfMRI-21', 'rsfMRI-22', 'rsfMRI-23', 'rsfMRI-24', 'tfMRI-1', 'tfMRI-2', 'tfMRI-5', 'tfMRI-COPE-1',
    'tfMRI-COPE-2', 'tfMRI-COPE-5', 'Summed Tracts', 'TBSS FA', 'TBSS ICVF', 'TBSS ISOVF', 'TBSS L1', 'TBSS L2',
    'TBSS L3', 'TBSS MD', 'TBSS MO', 'TBSS OD', 'FA', 'ICVF', 'ISOVF', 'L1', 'L2', 'L3', 'MD', 'MO', 'OD'
]
for idx, mod in enumerate(current_mods):
    vars_df.loc[vars_df["modality"] == mod, "modality"] = modality_names[idx]
    

In [12]:
ordered_cols = [
     'Modality',
     'bonf',
     'passes_bonf',
     'fdr',
     'passes_fdr',
     'idx',
     'names',
     'Categories',
     'pearson_r',
     'abs_pearson_r',
     't_test_statistic',
     'p_values',
     'p_values_corrected',
     'log_p_values',
     'log_p_values_corrected', 
]

for idx in range(len(vars_df)):
    if idx == 0:
        df = vars_df.iloc[idx].dataframe
        df['Modality'] = vars_df.iloc[idx].modality
        df['bonf'] = vars_df.iloc[idx].bonf
        df['fdr'] = vars_df.iloc[idx].fdr
        df['passes_bonf'] = df.log_p_values > df.bonf
        df['passes_fdr'] = df.log_p_values > df.fdr
        df = df[ordered_cols]
        df = df.drop(columns=['abs_pearson_r'])
    else:
        df2 = vars_df.iloc[idx].dataframe
        df2['Modality'] = vars_df.iloc[idx].modality
        df2['bonf'] = vars_df.iloc[idx].bonf
        df2['fdr'] = vars_df.iloc[idx].fdr
        df2['passes_bonf'] = df2.log_p_values > df2.bonf
        df2['passes_fdr'] = df2.log_p_values > df2.fdr
        df2 = df2[ordered_cols]
        df2 = df2.drop(columns=['abs_pearson_r'])
        df = pd.concat([df, df2])
        del df2

In [13]:
vID = []
for idx in range(len(df)):
    vID.append(varsVARS[np.where(varsHeader == df.iloc[idx].names)[0][0]])
df['Variable ID'] = vID
df

,Modality,bonf,passes_bonf,fdr,passes_fdr,idx,names,Categories,pearson_r,t_test_statistic,p_values,p_values_corrected,log_p_values,log_p_values_corrected,Variable ID
0,T1 Nonlinear,5.443326,False,4.755399,False,0,Ethnic background (0.0),Ethnic Background,0.014256,0.664910,0.506179,1.00000,0.295696,-0.000000,21000-0.0
1,T1 Nonlinear,5.443326,False,4.755399,False,1,Ethnic background (1.0),Ethnic Background,-0.029316,0.578459,0.563289,1.00000,0.249269,-0.000000,21000-1.0
2,T1 Nonlinear,5.443326,False,4.755399,False,2,Ethnic background (2.0),Ethnic Background,0.060473,1.530274,0.126445,0.97635,0.898099,0.010394,21000-2.0
3,T1 Nonlinear,5.443326,False,4.755399,False,3,Genotype measurement batch (0.0),Genetic Markers,-0.026032,1.203263,0.229008,1.00000,0.640149,-0.000000,22000-0.0
4,T1 Nonlinear,5.443326,False,4.755399,False,4,Heterozygosity (0.0),Genetic Markers,-0.010863,0.501956,0.615750,1.00000,0.210595,-0.000000,22003-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13872,OD,5.443326,False,4.904194,False,17504,Substances taken for depression (0.1),Mental Health,0.049207,0.890895,0.373641,1.00000,0.427546,-0.000000,20546-0.1
13873,OD,5.443326,False,4.904194,False,17506,Activities undertaken to treat depression (0.1),Mental Health,-0.059443,0.949049,0.343498,1.00000,0.464076,-0.000000,20547-0.1
13874,OD,5.443326,False,4.904194,False,17507,Manifestations of mania or irritability (0.1),Mental Health,-0.016164,0.268077,0.788841,1.00000,0.103011,-0.000000,20548-0.1
13875,OD,5.443326,False,4.904194,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.021930,0.892376,0.372321,1.00000,0.429082,-0.000000,21062-0.0


In [15]:
categories = sorted(list(set(df.Categories.to_list())))

first_df = True

for cat in categories:
    df_tmp = df[(df.Categories==cat) & (df.passes_bonf==True)].sort_values('log_p_values', ascending=False)
    df_tmp = df_tmp.head(10)
    if len(df_tmp) == 0:
        continue
    if first_df == True:
        df_print = df_tmp
        first_df = False
    else:
        df_print = pd.concat([df_print, df_tmp])
        
df_print = df_print.drop(columns=[
    'bonf', 'passes_bonf', 'fdr', 'passes_fdr', 'idx', 't_test_statistic', 'p_values',
    'p_values_corrected', 'log_p_values_corrected',
])

df_print_colums_order = [
    'Modality', 'log_p_values', 'pearson_r', 'Categories', 'names', 'Variable ID'
]

df_print = df_print[df_print_colums_order]

df_print = df_print.round(decimals=3)

df_print = df_print.rename(
    columns={
        'log_p_values' : '-log(p)',
        'pearson_r' : 'Correlation (r)',
        'Categories': 'Variable Category',
        'names' : 'Variable Description'
    }
)

df_print



,Modality,-log(p),Correlation (r),Variable Category,Variable Description,Variable ID
1725,TBSS FA,7.099,0.141,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1725,TBSS L1,6.717,0.137,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1725,TBSS OD,6.706,0.137,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1725,MD,6.631,0.136,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1686,L1,6.577,0.110,Alcohol,Alcohol intake frequency. (2.0),1558-2.0
1686,TBSS L1,6.439,0.109,Alcohol,Alcohol intake frequency. (2.0),1558-2.0
1725,L1,6.398,0.133,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1686,TBSS FA,6.335,0.108,Alcohol,Alcohol intake frequency. (2.0),1558-2.0
1684,MD,6.210,0.107,Alcohol,Alcohol intake frequency. (0.0),1558-0.0
1725,L2,6.209,0.131,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0


In [16]:
df_print.to_csv('Male_top_nIDPs.csv', index=False)

In [17]:
categories = sorted(list(set(df.Categories.to_list())))

first_df = True

for cat in categories:
    df_tmp = df[(df.Categories==cat) & (df.passes_bonf==True)].sort_values('log_p_values', ascending=False)
    df_tmp = df_tmp.head(5)
    if len(df_tmp) == 0:
        continue
    if first_df == True:
        df_print = df_tmp
        first_df = False
    else:
        df_print = pd.concat([df_print, df_tmp])
        
df_print = df_print.drop(columns=[
    'bonf', 'passes_bonf', 'fdr', 'passes_fdr', 'idx', 't_test_statistic', 'p_values',
    'p_values_corrected', 'log_p_values_corrected'
])

df_print_colums_order = [
    'Modality', 'log_p_values', 'pearson_r', 'Categories', 'names', 'Variable ID'
]

df_print = df_print[df_print_colums_order]

df_print = df_print.round(decimals=3)

df_print = df_print.rename(
    columns={
        'log_p_values' : '-log(p)',
        'pearson_r' : 'Correlation (r)',
        'Categories': 'Variable Category',
        'names' : 'Variable Description'
    }
)

df_print

df_print.to_csv('Male_top_nIDPs_reduced.csv', index=False)

In [18]:
df_print

,Modality,-log(p),Correlation (r),Variable Category,Variable Description,Variable ID
1725,TBSS FA,7.099,0.141,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1725,TBSS L1,6.717,0.137,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1725,TBSS OD,6.706,0.137,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1725,MD,6.631,0.136,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1686,L1,6.577,0.110,Alcohol,Alcohol intake frequency. (2.0),1558-2.0
4000,OD,7.038,-0.118,Blood Assays,IGF-1 (0.0),30770-0.0
4000,TBSS ICVF,6.616,-0.114,Blood Assays,IGF-1 (0.0),30770-0.0
4000,FA,6.411,-0.112,Blood Assays,IGF-1 (0.0),30770-0.0
4000,ISOVF,5.952,-0.108,Blood Assays,IGF-1 (0.0),30770-0.0
4000,L2,5.676,-0.105,Blood Assays,IGF-1 (0.0),30770-0.0


In [19]:
df

,Modality,bonf,passes_bonf,fdr,passes_fdr,idx,names,Categories,pearson_r,t_test_statistic,p_values,p_values_corrected,log_p_values,log_p_values_corrected,Variable ID
0,T1 Nonlinear,5.443326,False,4.755399,False,0,Ethnic background (0.0),Ethnic Background,0.014256,0.664910,0.506179,1.00000,0.295696,-0.000000,21000-0.0
1,T1 Nonlinear,5.443326,False,4.755399,False,1,Ethnic background (1.0),Ethnic Background,-0.029316,0.578459,0.563289,1.00000,0.249269,-0.000000,21000-1.0
2,T1 Nonlinear,5.443326,False,4.755399,False,2,Ethnic background (2.0),Ethnic Background,0.060473,1.530274,0.126445,0.97635,0.898099,0.010394,21000-2.0
3,T1 Nonlinear,5.443326,False,4.755399,False,3,Genotype measurement batch (0.0),Genetic Markers,-0.026032,1.203263,0.229008,1.00000,0.640149,-0.000000,22000-0.0
4,T1 Nonlinear,5.443326,False,4.755399,False,4,Heterozygosity (0.0),Genetic Markers,-0.010863,0.501956,0.615750,1.00000,0.210595,-0.000000,22003-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13872,OD,5.443326,False,4.904194,False,17504,Substances taken for depression (0.1),Mental Health,0.049207,0.890895,0.373641,1.00000,0.427546,-0.000000,20546-0.1
13873,OD,5.443326,False,4.904194,False,17506,Activities undertaken to treat depression (0.1),Mental Health,-0.059443,0.949049,0.343498,1.00000,0.464076,-0.000000,20547-0.1
13874,OD,5.443326,False,4.904194,False,17507,Manifestations of mania or irritability (0.1),Mental Health,-0.016164,0.268077,0.788841,1.00000,0.103011,-0.000000,20548-0.1
13875,OD,5.443326,False,4.904194,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.021930,0.892376,0.372321,1.00000,0.429082,-0.000000,21062-0.0


In [20]:
df = df.rename(
    columns={
        'log_p_values' : '-log(P)',
        'pearson_r' : 'Correlation (r)',
        'Categories': 'Variable Category',
        'names' : 'Variable Description',
        'bonf': 'Bonferroni Threshold',
        'passes_bonf': 'Passes Bonf.',
        'fdr': 'False Discovery Rate (FDR)',
        'passes_fdr': 'Passes FDR',
        'idx': 'Caterogy ID',
        't_test_statistic': 'T-Values',
        'p_values': 'P-Values',
        'p_values_corrected': 'FDR Adjusted P-Value',
        'log_p_values_corrected': 'FDR Adjusted -log(P)'
    }
)

df

,Modality,Bonferroni Threshold,Passes Bonf.,False Discovery Rate (FDR),Passes FDR,Caterogy ID,Variable Description,Variable Category,Correlation (r),T-Values,P-Values,FDR Adjusted P-Value,-log(P),FDR Adjusted -log(P),Variable ID
0,T1 Nonlinear,5.443326,False,4.755399,False,0,Ethnic background (0.0),Ethnic Background,0.014256,0.664910,0.506179,1.00000,0.295696,-0.000000,21000-0.0
1,T1 Nonlinear,5.443326,False,4.755399,False,1,Ethnic background (1.0),Ethnic Background,-0.029316,0.578459,0.563289,1.00000,0.249269,-0.000000,21000-1.0
2,T1 Nonlinear,5.443326,False,4.755399,False,2,Ethnic background (2.0),Ethnic Background,0.060473,1.530274,0.126445,0.97635,0.898099,0.010394,21000-2.0
3,T1 Nonlinear,5.443326,False,4.755399,False,3,Genotype measurement batch (0.0),Genetic Markers,-0.026032,1.203263,0.229008,1.00000,0.640149,-0.000000,22000-0.0
4,T1 Nonlinear,5.443326,False,4.755399,False,4,Heterozygosity (0.0),Genetic Markers,-0.010863,0.501956,0.615750,1.00000,0.210595,-0.000000,22003-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13872,OD,5.443326,False,4.904194,False,17504,Substances taken for depression (0.1),Mental Health,0.049207,0.890895,0.373641,1.00000,0.427546,-0.000000,20546-0.1
13873,OD,5.443326,False,4.904194,False,17506,Activities undertaken to treat depression (0.1),Mental Health,-0.059443,0.949049,0.343498,1.00000,0.464076,-0.000000,20547-0.1
13874,OD,5.443326,False,4.904194,False,17507,Manifestations of mania or irritability (0.1),Mental Health,-0.016164,0.268077,0.788841,1.00000,0.103011,-0.000000,20548-0.1
13875,OD,5.443326,False,4.904194,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.021930,0.892376,0.372321,1.00000,0.429082,-0.000000,21062-0.0


In [23]:
df.to_csv('Male_ALL_nIDPs.csv', index=False)

In [24]:
df_passing_FDR = df[df['Passes FDR']==True]

In [25]:
df_passing_FDR

,Modality,Bonferroni Threshold,Passes Bonf.,False Discovery Rate (FDR),Passes FDR,Caterogy ID,Variable Description,Variable Category,Correlation (r),T-Values,P-Values,FDR Adjusted P-Value,-log(P),FDR Adjusted -log(P),Variable ID
576,T1 Nonlinear,5.443326,False,4.755399,True,1155,Cereal intake (2.0),Diet,-0.096803,4.518118,6.576124e-06,0.029784,5.182030,1.526016,1458-2.0
1722,T1 Nonlinear,5.443326,True,4.755399,True,2869,Frequency of failure to fulfil normal expectat...,Alcohol,0.149588,4.796149,1.861965e-06,0.013281,5.730028,1.876780,20407-0.0
7373,T1 Nonlinear,5.443326,True,4.755399,True,10857,External causes - ICD10 (W014 - W01.4 Street a...,Medical History,-0.101721,4.775250,1.914054e-06,0.013281,5.718046,1.876780,41201-0.15435
12192,T1 Nonlinear,5.443326,False,4.755399,True,15729,Diagnoses - ICD10 (W014 - W01.4 Street and hig...,Medical History,-0.095083,4.460680,8.585159e-06,0.029784,5.066252,1.526016,41270-0.15435
1793,VBM,5.443326,True,4.996503,True,3019,Smoking status (0.0),Tobacco,0.099122,4.645602,3.593482e-06,0.016622,5.444485,1.779310,20116-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10498,MD,5.443326,False,4.164540,True,14008,Main speciality of consultant (recoded) (0.2),Medical History,-0.150449,4.162096,3.520048e-05,0.029773,4.453451,1.526179,41245-0.2
11012,MD,5.443326,False,4.164540,True,14549,Diagnoses - ICD10 (I10 - I10 Essential (primar...,Medical History,0.090720,4.254271,2.185951e-05,0.027735,4.660360,1.556978,41270-0.4052
752,OD,5.443326,True,4.904194,True,1572,Alcohol (2.0),Diet,0.198911,5.186594,2.861028e-07,0.001985,6.543478,2.702212,100022-2.0
1725,OD,5.443326,False,4.904194,True,2873,Frequency of drinking alcohol (0.0),Alcohol,0.119779,4.562332,5.491813e-06,0.025403,5.260284,1.595110,20414-0.0


In [26]:
df_passing_FDR.to_csv('Male_ALL_nIDPs_passing_FDR.csv', index=False)

In [27]:
!ls '../Analsysis Female New/'

1. MAEs, MSEs, PR + Saving Pandas Dataframes.ipynb
1.2. Train MAEs, MSEs, PR + Saving Pandas Dataframes.ipynb
1.3 Validation MAEs, MSEs, PR + Saving Pandas Dataframes.ipynb
10. ICA-Vars Correlations Analysis - Subject Direction.ipynb
10.1. ICA-Vars Correlations Analysis - Feature Direction.ipynb
10.2. ICA-Vars Correlations Analysis - Reduced Feature Direction.ipynb
11. IDP Predictions.ipynb
11.1. IDP Predictions - with Half B of Test Set.ipynb
11.2. IDP Predictions - MLP.ipynb
2. Correlation, Partial Correlation and Scatter Plots.ipynb
2.1. Correlation, Partial Correlation and Scatter Plots AFTER Deconfounding.ipynb
3. Averages across young and old subjects + Difference Maps + Location Plots.ipynb
4. Naive MAE-based Ensambling.ipynb
5. Multiple Linear Regression Ensembling.ipynb
5.1. ElasticNet & ElasticNet CV.ipynb
5.2. Repeated Elastic Net Technique.ipynb
5.3. PCA Variance & Weights for Ensembles.ipynb
5.4. MLP with CV Grid Search-Copy1.ipynb
5.4. MLP with CV Grid Search.ipynb
5.5. E

In [28]:
df_female_passing_FDR = pd.read_csv('../Analsysis Female New/Famale_ALL_nIDPs_passing_FDR.csv')

In [29]:
df_passing_FDR['Sex'] = 'Male'

/var/folders/66/_2ykm5fn1fs757m7q29p4fc40000gp/T/ipykernel_31119/1327222779.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_passing_FDR['Sex'] = 'Male'


In [30]:
df_female_passing_FDR['Sex'] = 'Female'

In [31]:
first_column = df_passing_FDR.pop('Sex')
df_passing_FDR.insert(0, 'Sex', first_column)

In [32]:
first_column = df_female_passing_FDR.pop('Sex')
df_female_passing_FDR.insert(0, 'Sex', first_column)

In [33]:
df_ALL_passing_FDR = pd.concat([df_female_passing_FDR, df_passing_FDR])

In [34]:
df_ALL_passing_FDR

,Sex,Modality,Bonferroni Threshold,Passes Bonf.,False Discovery Rate (FDR),Passes FDR,Caterogy ID,Variable Description,Variable Category,Variable ID,Correlation (r),T-Values,P-Values,FDR Adjusted P-Value,-log(P),FDR Adjusted -log(P)
0,Female,T1 Nonlinear,5.442009,False,3.767651,True,1209,"Never eat eggs, dairy, wheat, sugar (2.0)",Diet,6144-2.0,-0.079793,3.990406,6.786899e-05,0.020866,4.168329,1.680562
1,Female,T1 Nonlinear,5.442009,False,3.767651,True,3316,Heel broadband ultrasound attenuation (left) (...,Skeletal Measurements,4101-2.0,-0.087045,3.822675,1.362323e-04,0.036246,3.865720,1.440744
2,Female,T1 Nonlinear,5.442009,False,3.767651,True,3320,Speed of sound through heel (left) (2.0),Skeletal Measurements,4103-2.0,-0.086605,3.803203,1.473153e-04,0.038455,3.831752,1.415049
3,Female,T1 Nonlinear,5.442009,False,3.767651,True,3324,"Heel quantitative ultrasound index (QUI), dire...",Skeletal Measurements,4104-2.0,-0.089740,3.941984,8.371634e-05,0.023164,4.077190,1.635181
4,Female,T1 Nonlinear,5.442009,False,3.767651,True,3377,L1-L4 BMD (bone mineral density) (2.0),Skeletal Measurements,23204-2.0,-0.088822,4.129082,3.781682e-05,0.013080,4.422315,1.883396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10498,Male,MD,5.443326,False,4.164540,True,14008,Main speciality of consultant (recoded) (0.2),Medical History,41245-0.2,-0.150449,4.162096,3.520048e-05,0.029773,4.453451,1.526179
11012,Male,MD,5.443326,False,4.164540,True,14549,Diagnoses - ICD10 (I10 - I10 Essential (primar...,Medical History,41270-0.4052,0.090720,4.254271,2.185951e-05,0.027735,4.660360,1.556978
752,Male,OD,5.443326,True,4.904194,True,1572,Alcohol (2.0),Diet,100022-2.0,0.198911,5.186594,2.861028e-07,0.001985,6.543478,2.702212
1725,Male,OD,5.443326,False,4.904194,True,2873,Frequency of drinking alcohol (0.0),Alcohol,20414-0.0,0.119779,4.562332,5.491813e-06,0.025403,5.260284,1.595110


In [35]:
df_ALL_passing_FDR.to_csv('MALE_FEMALE_ALL_nIDPs_passing_FDR.csv', index=False)

In [36]:
df_female = pd.read_csv('../Analsysis Female New/Famale_ALL_nIDPs.csv')

In [37]:
df['Sex'] = 'Male'
df_female['Sex'] = 'Female'
first_column = df.pop('Sex')
df.insert(0, 'Sex', first_column)
first_column = df_female.pop('Sex')
df_female.insert(0, 'Sex', first_column)
df_ALL = pd.concat([df_female, df])
df_ALL

,Sex,Modality,Bonferroni Threshold,Passes Bonf.,False Discovery Rate (FDR),Passes FDR,Caterogy ID,Variable Description,Variable Category,Variable ID,Correlation (r),T-Values,P-Values,FDR Adjusted P-Value,-log(P),FDR Adjusted -log(P)
0,Female,T1 Nonlinear,5.442009,False,3.767651,False,0,Ethnic background (0.0),Ethnic Background,21000-0.0,0.000919,0.045977,0.963332,1.000000,0.016224,-0.000000
1,Female,T1 Nonlinear,5.442009,False,3.767651,False,1,Ethnic background (1.0),Ethnic Background,21000-1.0,-0.006902,0.142788,0.886524,1.000000,0.052309,-0.000000
2,Female,T1 Nonlinear,5.442009,False,3.767651,False,2,Ethnic background (2.0),Ethnic Background,21000-2.0,-0.006575,0.176541,0.859918,1.000000,0.065543,-0.000000
3,Female,T1 Nonlinear,5.442009,False,3.767651,False,3,Genotype measurement batch (0.0),Genetic Markers,22000-0.0,-0.072094,3.577774,0.000353,0.077575,3.451915,1.110277
4,Female,T1 Nonlinear,5.442009,False,3.767651,False,4,Heterozygosity (0.0),Genetic Markers,22003-0.0,-0.023098,1.143596,0.252903,1.000000,0.597046,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13872,Male,OD,5.443326,False,4.904194,False,17504,Substances taken for depression (0.1),Mental Health,20546-0.1,0.049207,0.890895,0.373641,1.000000,0.427546,-0.000000
13873,Male,OD,5.443326,False,4.904194,False,17506,Activities undertaken to treat depression (0.1),Mental Health,20547-0.1,-0.059443,0.949049,0.343498,1.000000,0.464076,-0.000000
13874,Male,OD,5.443326,False,4.904194,False,17507,Manifestations of mania or irritability (0.1),Mental Health,20548-0.1,-0.016164,0.268077,0.788841,1.000000,0.103011,-0.000000
13875,Male,OD,5.443326,False,4.904194,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,21062-0.0,-0.021930,0.892376,0.372321,1.000000,0.429082,-0.000000


In [38]:
df_ALL.to_csv('MALE_FEMALE_ALL_nIDPs.csv', index=False)